In [6]:
# Add Parent Folder to System Path

import os
import sys
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

####

import pandas as pd
import numpy as np

#Price originally in usd/m^3

df_fuel=pd.read_csv("../row_data/natural-gas-prices.csv")
df_countries=pd.read_csv("../row_data/Countries_ISO3.csv")
df_countries=pd.read_csv("../row_data/Countries_ISO3.csv")
df_fuel

,Entity,Code,Year,Gas - Prices
0,Crude Oil - OECD countries CIF (BP),NaN,1984,0.170605
1,Crude Oil - OECD countries CIF (BP),NaN,1985,0.162134
2,Crude Oil - OECD countries CIF (BP),NaN,1986,0.087832
3,Crude Oil - OECD countries CIF (BP),NaN,1987,0.105599
4,Crude Oil - OECD countries CIF (BP),NaN,1988,0.087420
...,...,...,...,...
230,Natural Gas - US Henry Hub (BP),NaN,2017,0.101018
231,Natural Gas - US Henry Hub (BP),NaN,2018,0.106336
232,Natural Gas - US Henry Hub (BP),NaN,2019,0.085709
233,Natural Gas - US Henry Hub (BP),NaN,2020,0.067852


In [7]:
#### HISTORICAL ######

countries=df_countries['Country Name'].values
ISO3=np.unique(df_countries['ISO3'].values)
years=np.unique(df_fuel['Year'].values)
coun_year=[]
for count in zip(countries,ISO3):
    for year in years:
        coun_year.append([count[0],count[1],year])
df_historical_out = pd.DataFrame(coun_year, columns=['Country','ISO3', 'Year'])
df_historical_out

,Country,ISO3,Year
0,Aruba,ABW,1984
1,Aruba,ABW,1985
2,Aruba,ABW,1986
3,Aruba,ABW,1987
4,Aruba,ABW,1988
...,...,...,...
10103,Zimbabwe,ZWE,2017
10104,Zimbabwe,ZWE,2018
10105,Zimbabwe,ZWE,2019
10106,Zimbabwe,ZWE,2020


In [8]:
df_fuel=pd.read_csv("../row_data/natural-gas-prices.csv")
df_fuel.set_index("Year", inplace = True)
# Using the operator .loc[]
# to select single row
average_prices=[]

for y in years:
    yp = np.average(df_fuel.loc[y]['Gas - Prices'])
    average_prices.append(yp)

average_prices=np.array(average_prices)
#1 MMBTU is equal to 27.3192 m3 
#(https://learnmetrics.com/mmbtu-to-m3-m3-to-mmbtu/#:~:text=1%20m3%20of%20natural%20gas%20is%20equal%20to%200.0366%20MMBTU.)
# therefore 1 usd/m^3=27.3192 m3/MMBTU * 1 usd/m^3 =27.3192 usd/MMBTU
prices=average_prices*27.3192
p=[]
for i in countries:
    p=np.append(p,prices)
df_historical_out['cost_enfu_fuel_natural_gas_usd_per_mmbtu']=p
df_historical_out.to_csv('../input_to_sisepuede/historical/cost_enfu_fuel_natural_gas_usd_per_mmbtu.csv')
df_historical_out

,Country,ISO3,Year,cost_enfu_fuel_natural_gas_usd_per_mmbtu
0,Aruba,ABW,1984,4.379788
1,Aruba,ABW,1985,4.424634
2,Aruba,ABW,1986,3.295098
3,Aruba,ABW,1987,2.794912
4,Aruba,ABW,1988,2.525073
...,...,...,...,...
10103,Zimbabwe,ZWE,2017,5.347339
10104,Zimbabwe,ZWE,2018,6.806364
10105,Zimbabwe,ZWE,2019,5.124565
10106,Zimbabwe,ZWE,2020,3.911051


In [10]:
#### PROJECTED ######

#Prediction for the next years will be the last historical value
years2=years
prices2=prices
for y in np.arange(min(years2),2051,1):
    if y not in years2:
        years2=np.append(years2,y)
        prices2=np.append(prices2,prices2[-1])

coun_year2=[]
for count in zip(countries,ISO3):
    for y_p in zip(years2,prices2):
        coun_year2.append([count[0],count[1],y_p[0],y_p[1]])
coun_year2

df_projected_out = pd.DataFrame(coun_year2, columns=['Country','ISO3', 'Year','cost_enfu_fuel_natural_gas_usd_per_mmbtu'])
df_projected_out.to_csv('../input_to_sisepuede/projected/cost_enfu_fuel_natural_gas_usd_per_mmbtu.csv')
df_projected_out

,Country,ISO3,Year,cost_enfu_fuel_natural_gas_usd_per_mmbtu
0,Aruba,ABW,1984,4.379788
1,Aruba,ABW,1985,4.424634
2,Aruba,ABW,1986,3.295098
3,Aruba,ABW,1987,2.794912
4,Aruba,ABW,1988,2.525073
...,...,...,...,...
17817,Zimbabwe,ZWE,2046,10.234523
17818,Zimbabwe,ZWE,2047,10.234523
17819,Zimbabwe,ZWE,2048,10.234523
17820,Zimbabwe,ZWE,2049,10.234523
